# Before running
## -This script has been created for extracting financial figures of Food Security Cluster within the appeal data recorded in FTS. 
## -No appeal data is not included. 
## -No credential is required. 
## -This script has been created in a Mac environment, there can be differences for other OSs.  

### Importing libraries

In [ ]:
from lxml import html
import requests
import pandas as pd
from urllib.request import urlopen
from bs4 import BeautifulSoup as BS
import os
from datetime import date
import json
import requests
import numpy as np

### Pre-defined information

In [ ]:
#Please indicate the ids of countries with more than 1 appeal for Food Security Cluster
morethanone_id = [1128]

#Please indicate the year of concern
year_list = [2023]

### Getting data from FTS webside through API and html 

In [ ]:
#get all sectors data to see total fund and requirements
df_all_sectors = pd.read_html('https://fts.unocha.org/appeals/overview/{}/plans'.format(year_list[0]))[1]

In [ ]:
#get the ids of appeals using api
df_analysis = pd.DataFrame()
for year in year_list:
    response = requests.get("https://api.hpc.tools/v1/public/plan/year/{}".format(year))
    dic_list = response.json()
    dic_list = dic_list['data']
    for i in range(0, len(dic_list)):
        df = pd.DataFrame([dic_list[i]['planVersion']])
        df_analysis = pd.concat([df_analysis, df])

#recent_analysis = df_analysis.sort_values(by='modified').drop_duplicates('country', keep='first')
all_ids = list(df_analysis['planId'])

In [ ]:
df_analysis.head()

import requests
response = requests.get("https://api.hpc.tools/v1/public/plan/year/2022")

dic_list = response.json()
dic_list = dic_list['data']
for i in range(0, len(dic_list)):
    df = pd.DataFrame([dic_list[i]['planVersion']])
    df_analysis = pd.concat([df_analysis, df])
all_ids = list(df_analysis['planId'])
return all_ids

In [ ]:
#functions for components
def get_requirement(row):
    for elem in row:
        if elem == '\n':
            row.remove(elem)            
    for i in range(0, len(row)):
        if row[i].get('headers')[0] == 'view-total-funding-table-column':
            fund_index = i
        elif row[i].get('headers')[0] == 'view-requirements-current-table-column':
            requirement_index = i
    requirement = row[requirement_index].text.strip()
    return requirement

def get_fund(row):
    for elem in row:
        if elem == '\n':
            row.remove(elem)            
    for i in range(0, len(row)):
        if row[i].get('headers')[0] == 'view-total-funding-table-column':
            fund_index = i
        elif row[i].get('headers')[0] == 'view-requirements-current-table-column':
            requirement_index = i
    fund = row[fund_index].text.strip()
    return fund

def get_sector(row):
    for elem in row:
        if elem == '\n':
            row.remove(elem)            
    sector = row[0].text.strip()
    return sector

#get the data from FTS webside
df = pd.DataFrame(columns = ['Appeal', 'Sector', 'Fund', 'Appeal_Code' ])

for i in all_ids:
    try:
        html = urlopen('https://fts.unocha.org/appeals/{}/clusters'.format(i))
        soup = BS(html)
        appeal = soup.title.text.strip().split('|')[0]
        appeal_code = i
        tables = soup.find_all('table')[1]
        if appeal_code in morethanone_id:
            row1 = soup.find(lambda tag: (tag.name=="td") and ('Agriculture' in tag.text)).parent.contents
            row2 = soup.find(lambda tag: (tag.name=="td") and ('Food' in tag.text)).parent.contents
            requirement1 = get_requirement(row1)
            requirement2 = get_requirement(row2)
            fund1 = get_fund(row1)
            fund2 = get_fund(row2)
            sector1 = get_sector(row1)
            sector2 = get_sector(row2)
            df = df.append({'Appeal': appeal,
                   'Sector': sector1,
                    'Current Requirement' : requirement1,
                   'Fund': fund1, 
                       'Appeal_Code': appeal_code}, ignore_index=True)
            df = df.append({'Appeal': appeal,
                   'Sector': sector2,
                    'Current Requirement' : requirement2,
                   'Fund': fund2, 
                       'Appeal_Code': appeal_code}, ignore_index=True)
        else:                     
            row = soup.find(lambda tag: (tag.name=="td") and (('Security' in tag.text) or ('Seguridad' in tag.text) or ('Sécurité' in tag.text) )).parent.contents
            requirement = get_requirement(row)
            fund = get_fund(row)
            sector = get_sector(row)
            df = df.append({'Appeal': appeal,
                   'Sector': sector,
                    'Current Requirement' : requirement,
                   'Fund': fund, 
                       'Appeal_Code': appeal_code}, ignore_index=True)        
    except:
        pass

In [ ]:
#get the data from FTS webside
df = pd.DataFrame(columns = ['Appeal', 'Sector', 'Fund', 'Appeal_Code' ])

for i in all_ids:
    try:
        html = urlopen('https://fts.unocha.org/appeals/{}/clusters'.format(i))
        soup = BS(html)
        appeal = soup.title.text.strip().split('|')[0]
        appeal_code = i
        tables = soup.find_all('table')[1]
        if appeal_code in morethanone_id:
            row1 = soup.find(lambda tag: (tag.name=="td") and ('Agriculture' in tag.text)).parent.contents
            row2 = soup.find(lambda tag: (tag.name=="td") and ('Food' in tag.text)).parent.contents
            requirement1 = get_requirement(row1)
            requirement2 = get_requirement(row2)
            fund1 = get_fund(row1)
            fund2 = get_fund(row2)
            sector1 = get_sector(row1)
            sector2 = get_sector(row2)
            df1 = pd.DataFrame({'Appeal': [appeal],
                   'Sector': [sector1],
                    'Current Requirement' : [requirement1],
                   'Fund': [fund1], 
                       'Appeal_Code': [appeal_code]})
            df2 = pd.DataFrame({'Appeal': [appeal],
                   'Sector': [sector2],
                    'Current Requirement' : [requirement2],
                   'Fund': [fund2], 
                       'Appeal_Code': [appeal_code]})
            df3 = pd.concat([df1,df2])
            
        else:                     
            row = soup.find(lambda tag: (tag.name=="td") and (('Security' in tag.text) or ('Seguridad' in tag.text) or ('Sécurité' in tag.text) )).parent.contents
            requirement = get_requirement(row)
            fund = get_fund(row)
            sector = get_sector(row)
            df_new = pd.DataFrame({'Appeal': [appeal],
                   'Sector': [sector],
                    'Current Requirement' : [requirement],
                   'Fund': [fund], 
                       'Appeal_Code': [appeal_code]})
            df = pd.concat([df,df_new])                    
    except:
        pass

df = pd.concat([df, df3])
df.reset_index(drop=True, inplace=True)

In [ ]:
#Arrange appeal names
hrp_list =['Humanitarian Response Plan', 'Plan de Respuesta Humanitario', 'Plan de Réponse Humanitaire']
fa_list = ['Flash Appeal']
rrp_list = ['Regional Refugee', 'Joint Response', 'Refugee and Migrant','Refugee Response' ]
other_list = ['Humanitarian Needs and Priorities', 'Floods', 'Emergency']

# create a list of our conditions
conditions = [
    (df['Appeal'].apply(lambda x: any(a in x for a in hrp_list))),
    (df['Appeal'].apply(lambda x: any(a in x for a in fa_list))),
    (df['Appeal'].apply(lambda x: any(a in x for a in rrp_list))),
    (df['Appeal'].apply(lambda x: any(a in x for a in other_list)))]

# create a list of the values we want to assign for each condition
values = ['HRP', 'Flash Appeal', 'RRP', 'Other']

# create a new column and use np.select to assign values to it using our lists as arguments
df['Plan Type'] = np.select(conditions, values)
df['Plan Type'] = np.where(df['Plan Type']=='0', 'Other', df['Plan Type'])

In [ ]:
#Add the country name from appeal list if it is not already here
country_list = ['Afghanistan', 'Bangladesh:', 'Burkina', 'Burundi',
       'Cameroon', 'Centrafricaine', 'African',  'Tchad', 'Chad', 'Colombia', 
       'Congo', 'Salvador', 'Ethiopia', 'Guatemala', 'Haiti', 'Honduras',
       'Iraq', 'Kenya', 'Lebanon', 'Libya', "Madagascar's", 'Madagascar', 'Mali', 'Mozambique', 'Madagascar', 'Malawi',
       'Myanmar', 'Niger', 'Nigeria', 'Pakistan', 'occupied', 'Occupied', 'Somalia',
       'South', 'Sudan', 'Syrian','Syria', 'Ukraine', 'Venezuela', 'Yemen',
       'Zimbabwe', 'SouthSudan', 'Rohingya', 'Türkiye']

In [ ]:
#add year column to validate the year
df['Year'] = df['Appeal'].apply(lambda x: x.split()[-1])
#Venezuela has RMRP instead of year, so we will add it manually for 2022.
#df['Year'] = df['Year'].replace('(RMRP)', 2022)
pd.set_option('display.max_rows', 999)

In [ ]:
#Correct the country names to be consistent with FSC files
df['Country'] = df['Appeal'].apply(lambda i: ''.join([x for x in list(i.split()) if x in country_list]))
df = df.drop(df[df['Country'] == ''].index).reset_index(drop=True)
country_dict = {'Afghanistan':'Afghanistan', 
                       'Bangladesh:': 'Bangladesh', 
                       'Burkina': 'Burkina Faso', 
                       'Burundi': 'Burundi', 
                       'Cameroon':'Cameroon',
                       'Centrafricaine':'CAR', 
                       'African': 'CAR', 
                       'Tchad':'Chad', 
                       'Chad': 'Chad',
                       'Colombia':'Colombia', 
                       'Congo':'DRC',
                       'Salvador': 'El Salvador',
                       'Ethiopia':'Ethiopia', 
                       'Guatemala':'Guatemala', 
                       'Haiti':'Haiti', 
                       'Honduras':'Honduras', 
                       'Iraq':'Iraq',
                       'Lebanon':'Lebanon', 
                       'Libya':'Libya', 
                       "Madagascar's":'Madagascar', 
                       'Madagascar': 'Madagascar',
                       'Mali': 'Mali',
                       'Mozambique':'Mozambique', 
                       'Myanmar':'Myanmar', 
                       'Niger':'Niger',
                       'Nigeria':'Nigeria', 
                       'Pakistan':'Pakistan',
                       'Rohingya':'Bangladesh',
                       'occupied':'Palestine',
                       'Occupied':'Palestine',
                       'Somalia':'Somalia', 
                       'SouthSudan':'South Sudan',
                       'South':'South Sudan', 
                       'Sudan':'Sudan', 
                       'Syria':'Syria', 
                       'Syrian': 'Syria',
                       'Türkiye': 'Türkiye',
                       'Ukraine':'Ukraine', 
                       'Venezuela,':'Venezuela', 
                       'Yemen':'Yemen', 
                       'Zimbabwe':'Zimbabwe'}
df['Country'].replace(country_dict, inplace=True)



In [ ]:
df

In [ ]:
#Convert string to intiger with removing commas and dolar signs
char_to_replace = {',': '', '$': ''}
df['Fund Received'] = df['Fund'].apply(lambda a: int(a.translate(str.maketrans(char_to_replace))))
df['Current_Requirement'] = df['Current Requirement'].apply(lambda a: int(a.translate(str.maketrans(char_to_replace))))

#reorder the columns
df = df[[ 'Country', 'Appeal', 'Plan Type', 'Sector', 'Year', 'Appeal_Code', 'Current_Requirement', 'Fund Received']]
    
#get rid off Countries you dont need
df = df.drop(df[df['Country']==''].index)

In [ ]:
df['Appeal'] = df['Appeal'].apply(lambda x: x[:-1])
df_all_sectors['Country'] = df_all_sectors['Appeal/Plan  Sort descending'].apply(lambda i: ''.join([x for x in list(i.split()) if x in country_list]))
df_all_sectors['Country'].replace(country_dict, inplace=True)
df_all_sectors['Plan type'].replace({'Humanitarian response plan': 'HRP',
'Regional response plan': 'RRP',
'Flash appeal': 'Flash Appeal'}, inplace=True)
df_all_sectors = df_all_sectors.merge(df[['Appeal', 'Appeal_Code']], how='left', left_on='Appeal/Plan  Sort descending', right_on = 'Appeal')
df_all_sectors.drop('Appeal', axis=1, inplace=True)


In [ ]:
path = os.getcwd()
today = str(date.today())

In [ ]:
df_all_sectors.insert(0, 'Date', today)

## Control Check for Appeals and Countries

In [ ]:
if len(df['Appeal']) < 44:
    print('!!!WARNING!!!!  CHECK THE NUMBER OF APPEALS')
if len(df['Country'].unique()) < 36:
    print('!!!WARNING!!!!  CHECK THE NUMBER OF COUNTRY')

In [ ]:
# Create a Pandas Excel writer using XlsxWriter as the engine.
writer = pd.ExcelWriter('FTS_Live_2023.xlsx', engine='xlsxwriter')

# Write each dataframe to a different worksheet.
df.to_excel(writer, sheet_name='Sheet1', index=False)
df_all_sectors.to_excel(writer, sheet_name='All_Sectors', index=False)

# Close the Pandas Excel writer and output the Excel file.
writer.save()

In [ ]:
#export to excel
df.to_excel('FTS_{}.xlsx'.format(today), index=False)